# Simple probability Monte Carlo style

A media firm identify that 85 per cent of users of their app are using it to read sports related news articles. Out of a random sample of 100 users, find the probability that exactly three quarters of the sample are reading sports articles. Give your answer in the form d.dddd (so four significant digits).

In [52]:
52/4

13.0

In [2]:
import pandas as pd
import numpy as np
import pymc as pm

In [12]:
np.random.binomial(5,0.8,size=100).shape

(100,)

In [49]:
def sample_users(n_users,p,n_tries=100000):
    return np.random.binomial(n_users,p,size=int(n_tries))
samples = sample_users(100,0.8,n_tries=int(1e9))

In [50]:
len(samples[samples==75])/len(samples)

MemoryError: 

In [48]:
len(samples[samples==75])/len(samples)

0.04388304

# Q2

Each Pivigo team is composed of 4 people drawn randomly from a pool of 30 women and 60 men. What is the probability that the teams are composed of: two women and two men (give your answer to two significant digits)

In [28]:
def sample_team(n_team=4,n_samples=1000):
    total_people = np.hstack([np.ones(30),np.zeros(60)])
    samples = np.array([np.random.choice(total_people,n_team,replace=False) for i in range(n_samples)])
    return samples

In [36]:
teams = sample_team(n_samples=1000000)

In [43]:
women_per_team = teams.sum(axis=1)
np.round(len(women_per_team[women_per_team==2])/len(women_per_team),3)#answer

0.30199999999999999

# Q4

Four men

In [44]:
women_per_team = teams.sum(axis=1)
np.round(len(women_per_team[women_per_team==0])/len(women_per_team),3)#answer

0.191

# Q3

At least one women

In [45]:
women_per_team = teams.sum(axis=1)
np.round(len(women_per_team[women_per_team>=1])/len(women_per_team),3)#answer

0.80900000000000005

# classification and conditional probability

13 cards per suit

In [61]:
from collections import Counter

In [65]:
def draw_samples(n_cards,n_tries):
    cards = ['A']*13+['B']*13+['C']*13+['D']*13
    return np.array([np.random.choice(cards,size=n_cards,replace=False) for i in range(n_tries)])

In [69]:
def one_spade_one_heart(samples):
    nice = []
    for i in range(samples.shape[0]):
        c = Counter(samples[i])
        if c['A'] ==1 and c['B']==1:
            nice.append(True)
    return len(nice)/samples.shape[0]

In [70]:
samples = draw_samples(2,1000000)
one_spade_one_heart(samples)

0.127714

In [67]:
t = Counter(draw_samples(10,100)[0])

In [68]:
t['A']

2

In [81]:
np.random.permutation(24)

array([ 0, 11, 17,  8,  3, 20, 14,  4, 13,  1, 23,  6, 12, 22, 21, 19,  2,
        9,  7,  5, 15, 18, 10, 16])

In [55]:
12/51

0.23529411764705882

In [73]:
import pymc3 as pm

N = 100
with pm.Model() as model:
    p = pm.Uniform("freq_cheating", 0, 1)
with model:
    true_answers = pm.Bernoulli("truths", p, shape=N, testval=np.random.binomial(1, 0.5, N))
with model:
    first_coin_flips = pm.Bernoulli("first_flips", 0.5, shape=N, testval=np.random.binomial(1, 0.5, N))
print(first_coin_flips.tag.test_value)

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


[1 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 0 1
 1 0 1 0 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0 0 0 1
 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0]


/home/kalidus/anaconda3/lib/python3.5/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


In [74]:
with model:
    second_coin_flips = pm.Bernoulli("second_flips", 0.5, shape=N, testval=np.random.binomial(1, 0.5, N))



/home/kalidus/anaconda3/lib/python3.5/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


In [75]:
import theano.tensor as tt
with model:
    val = first_coin_flips*true_answers + (1 - first_coin_flips)*second_coin_flips
    observed_proportion = pm.Deterministic("observed_proportion", tt.sum(val)/float(N))

In [80]:
0.5*(1/3)*3

0.5

In [54]:
13/52*13/51

0.06372549019607843

You are about to get on a train, you know it rains 50 per cent of the time where you are going, and you want to know if you should bring an umbrella. You call 3 random friends of yours who live there and ask each independently if it is raining. Each of your friends has a 2/3 chance of telling you the truth and a 1/3 chance of messing with you by lying. All 3 friends tell you: “Yes, it is raining”. What is the probability that it is actually raining? Give the answer to two significant digits